# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [48]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas 
import requests

# Import API key
from api_keys import geoapify_key

In [49]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,masuguru,-11.3667,38.4167,21.87,98,70,1.15,TZ,2024-04-24
1,1,edinburgh of the seven seas,-37.0676,-12.3116,15.28,74,59,3.33,SH,2024-04-24
2,2,tiksi,71.6872,128.8694,-10.00,80,98,3.75,RU,2024-04-24
3,3,guerrero negro,27.9769,-114.0611,15.80,76,85,7.16,MX,2024-04-24
4,4,ad dawadimi,24.5077,44.3924,15.64,22,2,2.59,SA,2024-04-24


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:
%%capture --no-display

# Configure the map plot
full_map = city_data_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True, 
    tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "Country",
    hover_cols = ["City", "Country"]
)

# Display the map
full_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [55]:
# Narrow down cities that fit criteria and drop any results with null values
optimal_city_data_df = city_data_df[(city_data_df['Max Temp'] > 25) & # Not too cold
                                    (city_data_df['Max Temp'] < 30) & # Not too hot
                                    (city_data_df['Humidity'] < 85) & # reasonable humidity
                                    (city_data_df['Lat'] < 0)] # only locations in southern hemisphere

# Drop any rows with null values
optimal_city_data_df = optimal_city_data_df.dropna()

# Display sample data
optimal_city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,sao jose da coroa grande,-8.8978,-35.1478,26.23,83,57,1.63,BR,2024-04-24
58,58,pitimbu,-7.4706,-34.8086,26.33,81,94,2.82,BR,2024-04-24
85,85,bandar lampung,-5.4254,105.2580,29.76,71,97,1.59,ID,2024-04-24
95,95,bengkulu,-3.8004,102.2655,27.99,75,100,0.19,ID,2024-04-24
140,140,alofi,-19.0595,-169.9187,28.94,74,100,2.57,NU,2024-04-24


### Step 3: Create a new DataFrame called `hotel_df`.

In [56]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = optimal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

/var/folders/2v/bzzw9p052_79xpvbbrhhbtzw0000gn/T/ipykernel_19750/3131284808.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
30,sao jose da coroa grande,BR,-8.8978,-35.1478,83,
58,pitimbu,BR,-7.4706,-34.8086,81,
85,bandar lampung,ID,-5.4254,105.2580,71,
95,bengkulu,ID,-3.8004,102.2655,75,
140,alofi,NU,-19.0595,-169.9187,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [58]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel" #hotels
conditions = "internet_access" #with internet access

params = {
    "categories":categories,
    "conditions":conditions,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row['Lat']
    longitude=row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data

hotel_df

Starting hotel search
sao jose da coroa grande - nearest hotel: Pousada Vivenda Oriente
pitimbu - nearest hotel: No hotel found
bandar lampung - nearest hotel: Hotel Anugerah Express Lampung
bengkulu - nearest hotel: Wedika Hotel
alofi - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
port mathurin - nearest hotel: Oasis Vacances
kone - nearest hotel: Pacifik Appartels
fale old settlement - nearest hotel: No hotel found
avarua - nearest hotel: Natura Nikao Beach Bungalow
labasa - nearest hotel: Riverview Private Hotel
yaren - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
mooloolaba - nearest hotel: Alex Seaside Resort
uturoa - nearest hotel: Te Ava Piti Lodge
yacuiba - nearest hotel: residencial Dieguito
port douglas - nearest hotel: Motel Port Douglas
carnarvon - nearest hotel: No hotel found
geraldton - nearest hotel: The Gerald Apartment Hotel
lakatoro 

,City,Country,Lat,Lng,Humidity,Hotel Name
30,sao jose da coroa grande,BR,-8.8978,-35.1478,83,Pousada Vivenda Oriente
58,pitimbu,BR,-7.4706,-34.8086,81,No hotel found
85,bandar lampung,ID,-5.4254,105.2580,71,Hotel Anugerah Express Lampung
95,bengkulu,ID,-3.8004,102.2655,75,Wedika Hotel
140,alofi,NU,-19.0595,-169.9187,74,No hotel found
155,atafu village,TK,-8.5421,-172.5159,73,No hotel found
156,west island,CC,-12.1568,96.8225,74,No hotel found
175,ta`u,AS,-14.2336,-169.5144,70,No hotel found
183,port mathurin,MU,-19.6833,63.4167,77,Oasis Vacances
186,kone,NC,-21.0595,164.8658,73,Pacifik Appartels


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points (
    "Lng",
    "Lat",
    geo = True, 
    tiles='OSM',
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "Country",
    hover_cols = ["Hotel Name", "City", "Country"]
)
# Display the map

hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name,City)